In [1]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import chi2

In [2]:
pd.set_option('mode.chained_assignment', None)
pd.options.mode.copy_on_write = True
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
print(f'Current working directory is {os.getcwd()}')

Current working directory is C:\Users\cavin\Documents\NSS_Projects\ACCR-project-clj\accre-carbonara\notebooks


In [4]:
# def ce5_func():
    # read in dataset 
    # ce5 = pd.read_csv('../data/slurm_wrapper_ce5.log',
                      # header=None,
                      # delimiter=' - ',
                      # engine='python',
                    #   nrows=1000000
                      # )
    
    # set column names
    # colnames = ['END', 'USER', 'RETRY', 'TIME', 'RETURNCODE', 'COMMAND']
    # ce5.columns = colnames
    
    # Clean and convert other columns
    # ce5['TIME'] = ce5['TIME'].str.replace('time', '', regex=False).astype(float)
    # ce5['USER'] = ce5['USER'].str.replace('user', '', regex=False).astype(int)
    # ce5['RETRY'] = ce5['RETRY'].str.replace('retry', '', regex=False).astype(int)
    # ce5['RETURNCODE'] = ce5['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
    # ce5['COMMAND'] = ce5['COMMAND'].str.replace('command', '', regex=False)
    # ce5['COMMAND'] = [re.sub("[\'\\[\\] ]", '', row).split(',') for row in ce5['COMMAND']];
    
    # slicing out the records that satisfy unresponsive conditions outlined. 
    # ce5_filtered = ce5[(ce5['USER'] == 9204) & (ce5['RETURNCODE'] == 1.0) & (ce5['TIME'] > 15)]
    # ce5_unresponsive_df = ce5_filtered[ce5_filtered['COMMAND'].map(lambda x: any('sbatch' in cmd for cmd in x))]
    # return ce5_unresponsive_df

In [5]:
# def ce6_func():
    # read in dataset 
    # ce6 = pd.read_csv('../data/slurm_wrapper_ce6.log',
                      # header=None,
                      # delimiter=' - ',
                      # engine='python',
                    #   nrows=10000
                      # )
    # set column names
    # colnames = ['END', 'USER', 'RETRY', 'TIME', 'RETURNCODE', 'COMMAND']
    # ce6.columns = colnames
    
    # Clean and convert other columns
    # ce6['TIME'] = ce6['TIME'].str.replace('time', '', regex=False).astype(float)
    # ce6['USER'] = ce6['USER'].str.replace('user', '', regex=False).astype(int)
    # ce6['RETRY'] = ce6['RETRY'].str.replace('retry', '', regex=False).astype(int)
    # ce6['RETURNCODE'] = ce6['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
    # ce6['COMMAND'] = ce6['COMMAND'].str.replace('command', '', regex=False)
    # ce6['COMMAND'] = [re.sub("[\'\\[\\] ]", '', row).split(',') for row in ce6['COMMAND']];
    
    # slicing out the records that satisfy unresponsive conditions outlined. 
    # ce6_filtered = ce6[(ce6['USER'] == 9204) & (ce6['RETURNCODE'] == 1.0) & (ce6['TIME'] > 15)]
    # ce6_unresponsive_df = ce6_filtered[ce6_filtered['COMMAND'].map(lambda x: any('sbatch' in cmd for cmd in x))]
    # return ce6_unresponsive_df

In [6]:
def jobs_ce56_func():
    # Import data
    jobs = pd.read_csv("../data/fullsample.csv",
                    # nrows = 1000000
                    )
    ce5 = pd.read_csv('../data/ce5_unresponsive.csv') # ce5_func
    ce6 = pd.read_csv('../data/ce6_unresponsive.csv') # ce6_func
    jobs = jobs[jobs['END'] != 'Unknown']
    
    # Convert dates to datetime objects
    jobs['END'] = pd.to_datetime(jobs['END'])
    jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'])
    
    # String manipulation of memory columns
    jobs['USEDMEM'] = jobs['USEDMEM'].str[:-1]
    jobs.insert(5, 'Mc_Mn', jobs['REQMEM'].str[-2:])
    jobs['REQMEM']  = jobs['REQMEM'].str[:-2]
    
    # Convert to numeric values for easier manipulation, and create difference col
    jobs['USEDMEM'] = pd.to_numeric(jobs['USEDMEM'])
    jobs['REQMEM'] = pd.to_numeric(jobs['REQMEM'])
    jobs['DIFFMEM'] = jobs['REQMEM'] - jobs['USEDMEM']
    
    # Create boolean column for status = COMPLETE, drop all rows where status is not complete
    jobs.insert(2, 'COMPLETE', jobs['STATE'] == 'COMPLETED')
    jobs['COMPLETE'] = jobs['COMPLETE'].astype(int)
    jobs = jobs.drop(columns=['STATE'])
    jobs = jobs[jobs['COMPLETE'] != 0]
    
    # Create columns to assign failure, job counts, and concat into one df
    ce56 = pd.concat([ce5, ce6])
    jobs['FAILED'] = 0
    jobs['JOBCOUNT'] = 1
    ce56['JOBCOUNT'] = 0
    ce56['FAILED'] = 1
    jobs_ce56 = pd.concat([jobs, ce56])
    
    # Broadened to 5 minute intervals in desired datetime column 'END' 
    jobs_ce56['END'] = pd.to_datetime(jobs_ce56['END']).dt.floor('5min')
    # ce56.to_csv('../data/ce56.csv', index=False)
    return jobs_ce56

In [7]:
jobs_ce56_func();

In [18]:
def reg_df_func():
    # Narrow down DataFrame to columns that matter
    df = jobs_ce56_func()[['END', 'JOBID', 'FAILED', 'USEDMEM']]
    reg_df = df.groupby('END').agg({'FAILED':'sum', 'JOBID': 'count', 'USEDMEM': 'mean'}).reset_index()
    reg_df = reg_df.rename(columns={'END': 'time', 'FAILED': 'slurm_crashes', 'JOBID': 'completed_jobs', 'USEDMEM': 'used_mem'})
    reg_df['fails'] = [1 if value > 0 else 0 for value in reg_df['slurm_crashes']]
    return reg_df

In [19]:
reg_df_func()

,time,slurm_crashes,completed_jobs,used_mem,fails
0,2020-10-01 00:10:00,0,3,363.32,0
1,2020-10-01 00:15:00,0,3,489.28,0
2,2020-10-01 00:20:00,0,9,6.23,0
3,2020-10-01 00:25:00,0,18,77.65,0
4,2020-10-01 00:30:00,0,11,6.23,0
...,...,...,...,...,...
105489,2021-10-07 20:10:00,0,1,0.09,0
105490,2021-10-07 20:15:00,0,1,0.09,0
105491,2021-10-07 20:25:00,0,1,393.07,0
105492,2021-10-07 20:30:00,0,1,0.09,0


In [20]:
reg_df = reg_df_func()

In [21]:
reg_df[reg_df['fails'] == 1]

,time,slurm_crashes,completed_jobs,used_mem,fails
4969,2020-10-18 06:15:00,1,4,2354.53,1
4973,2020-10-18 06:35:00,1,5,9831.25,1
4976,2020-10-18 06:50:00,2,60,259.12,1
4987,2020-10-18 07:45:00,3,14,1417.35,1
4988,2020-10-18 07:50:00,1,7,1198.43,1
...,...,...,...,...,...
101949,2021-09-24 18:10:00,1,32,10928.49,1
101961,2021-09-24 19:10:00,1,26,5821.96,1
104131,2021-10-02 08:10:00,1,35,1389.87,1
104254,2021-10-02 18:25:00,1,59,1159.08,1


In [22]:
reg_df[reg_df['time'] == '2020-10-18 07:45:00']

,time,slurm_crashes,completed_jobs,used_mem,fails
4987,2020-10-18 07:45:00,3,14,1417.35,1


In [23]:
model = smf.logit('fails ~ completed_jobs',
                    data = reg_df).fit()

Optimization terminated successfully.
         Current function value: 0.103913
         Iterations 8


In [24]:
model.params

Intercept        -3.80
completed_jobs   -0.00
dtype: float64

In [25]:
def logistic(x):
    return 1 / (1 + np.exp(-x))

In [28]:
jobs = 300

logit_p = model.params['Intercept'] + model.params['completed_jobs']*jobs

print(f'Estimated Probability: {logistic(logit_p)}')

Estimated Probability: 0.02019343225352652


In [29]:
slim_model = smf.logit('fails ~ 1', data = reg_df).fit()
full_model = smf.logit('fails ~ slurm_crashes + completed_jobs', data = reg_df).fit()

Optimization terminated successfully.
         Current function value: 0.103925
         Iterations 8


C:\Users\cavin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\cavin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\cavin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\cavin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np

         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix

In [ ]:
G2 = -2 * (slim_model.llf - full_model.llf)
G2

In [ ]:
df3 = full_model.df_model - slim_model.df_model
df3

In [ ]:
p = chi2.sf(G2, df = df3)
p

In [ ]:
lm = smf.ols('completed_jobs ~ slurm_crashes', data = reg_df).fit()

In [ ]:
jobs = 12

lm.params['Intercept'] + lm.params['slurm_crashes'] * jobs

In [ ]:
reg_df['rolling'] = reg_df.rolling(7)['slurm_crashes'].sum()

In [ ]:
reg_df.head()

In [ ]:
new_lm = smf.ols('completed_jobs ~ rolling', data = reg_df).fit()

In [ ]:
reg_df['gage_shifted'] = reg_df['completed_jobs'].shift(periods = 1)

In [ ]:
shift_lm = smf.ols('completed_jobs ~ gage_shifted', data = reg_df).fit()

In [ ]:
jobs = 12

shift_lm.params['Intercept'] + shift_lm.params['gage_shifted'] * jobs

In [ ]:
sm.stats.anova_lm(lm)

In [ ]:
filtered_reg_df = reg_df.iloc[:len(lm.fittedvalues)]

filtered_reg_df.plot(x='slurm_crashes', y='completed_jobs', kind='scatter')
plt.plot(filtered_reg_df['slurm_crashes'], lm.fittedvalues, color='black')